In [1]:
import tflearn
from tflearn.data_utils import *

In [10]:
maxlen = 25

path = "output.txt"
string_utf8 = open(path, "r").read()

In [12]:
def processData(string, seq_maxlen=25, char_idx=None):

    if char_idx is None:
        char_idx = chars_to_dictionary(string)

    len_chars = len(char_idx)

    sequences = []
    next_chars = []

    for i in range(0, len(string) - seq_maxlen - 1):
        sequences.append(string[i:i+seq_maxlen])
        next_chars.append(string[i+1:i+seq_maxlen+1])

    X = np.zeros((len(sequences), seq_maxlen, len_chars), dtype=np.bool)
    Y = np.zeros((len(sequences), seq_maxlen, len_chars), dtype=np.bool)

    for i, seq in enumerate(sequences):
        for t, char in enumerate(seq):
            X[i, t, char_idx[char]] = 1
            Y[i, t, char_idx[next_chars[i][t]]] = 1

    return X, Y, char_idx

In [13]:
def sample(a, temperature):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [14]:
def generate(model, seq_length, maxlen, temperature, char_idx_dic, idx_char_dic, seq_seed=None):

    generated = seq_seed[:]
    sequence = seq_seed[:]
    whole_sequence = seq_seed[:]

    for i in range(seq_length):
        x = np.zeros((1, maxlen, len(char_idx_dic)))
        for t, char in enumerate(sequence):
            x[0, t, char_idx_dic[char]] = 1.

        preds = model.predict(x)[0][-1]
        next_index = sample(preds, temperature)
        next_char = idx_char_dic[next_index]

        generated += next_char
        sequence = sequence[1:] + next_char
        whole_sequence += next_char
        
    return whole_sequence

In [15]:
maxlen = 25
X, Y, char_idx = \
    processData(string_utf8, seq_maxlen=maxlen)

In [16]:
X.shape

(129945, 25, 82)

In [18]:
import tensorflow as tf

tf.reset_default_graph()
yp = tf.placeholder(shape=[None, maxlen, len(char_idx)], dtype=tf.float32)                       

g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.time_distributed(g, tflearn.fully_connected, [len(char_idx)])               
g = tflearn.softmax(g)                                                      
g = tflearn.reshape(g, [-1, maxlen, len(char_idx)])   

g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001,placeholder=yp)


model = tflearn.DNN(g, tensorboard_verbose=0)

In [20]:
model.fit(X, Y, n_epoch=10,validation_set=0.1, show_metric=True,
          batch_size=64,run_id='talyor_swift')

Training Step: 36559  | total loss: 0.56599 | time: 100.720s
| Adam | epoch: 020 | loss: 0.56599 - acc: 0.3928 -- iter: 116928/116950
Training Step: 36560  | total loss: 0.54941 | time: 104.709s
| Adam | epoch: 020 | loss: 0.54941 - acc: 0.3910 | val_loss: 0.46943 - val_acc: 0.4010 -- iter: 116950/116950
--


In [21]:
idx_char_dic = {v: k for k, v in char_idx.items()}

In [40]:
'''
Randomly generate a short piece of lyrics as starting point
'''
seed = random_sequence_from_string(string_utf8, maxlen)
seed

' you tell me what more do'

In [46]:
output_seq = generate(model, 2000, maxlen, 0.85, char_idx, idx_char_dic, seed)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
output_seq

' you tell me what more do I need\nAnd they\'ll be chasing me this long\nBaby but I figure it out\nDon\'t know what\'s down this road, I\'m just walked by\nBut you had to talk about the good times, you and I\n\'Cause, baby, now we got bad blood\nHey\nAll is quiet in the world to let it go free\nAnd this love came back down\nTalk dark and be us the should step in\nAnd force us in a simple way\nIf I don\'t know if I should.\nI knew his world moved too fast and burned too free\nAnd this love came back to me\nThis love left a permanent mark\nThis love is glowing in the dark\nThese hands had to let it go free\nAnd this love is glowing in the dark\nThese hands had to let it go free\nAnd this love came back down\nLa, la, la\nWhen you come back down\nLa, la, la\nWhen you come back down\nLa, la, la\nWhen you come back down\nLa, la, la\nWhen you come back down\nLa, la,\nHe can\'t keep his wild eyes on the drought was the very worst\nWhen the flowers through his hair\nI\'m laughing Last forever a

In [ ]:
import codecs

file = codecs.open("talyor swift.txt", "w", "utf-8")
file.write(output_seq)
file.close()